In [71]:
import numpy as np
import pandas as pd

In [72]:
txn = pd.read_csv(
    'train_transaction.csv',
    usecols=['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'addr1']
)

In [76]:
txn.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,addr1,transaction_time
0,2987000,86400,68.5,W,13926,315.0,1970-01-02 00:00:00
1,2987001,86401,29.0,W,2755,325.0,1970-01-02 00:00:01
2,2987002,86469,59.0,W,4663,330.0,1970-01-02 00:01:09
3,2987003,86499,50.0,W,18132,476.0,1970-01-02 00:01:39
4,2987004,86506,50.0,H,4497,420.0,1970-01-02 00:01:46


Data Cleaning:

1. Convert Date Colume to Datetime Object

In [74]:
txn['transaction_time'] = pd.to_datetime(txn['TransactionDT'], unit='s')

2. Remove duplicate transaction ids

In [75]:
txn.drop_duplicates(subset='TransactionID', inplace = True)

3. Handling Missing Values

In [77]:
txn['addr1'].isna().sum()

np.int64(65706)

In [78]:
txn['addr1']= txn['addr1'].fillna(-1)

In [79]:
txn['addr1_missing'] = (txn['addr1'] == -1).astype(int)

In [80]:
txn = txn.sort_values('transaction_time')

In [81]:
txn = txn.drop(columns=['TransactionDT'])

Feature Engineering


In [82]:
txn['time_diff']= (txn.groupby('card1')['transaction_time'].diff().dt.total_seconds())
txn['txn_per_hour'] = 3600/ txn['time_diff']

In [83]:
txn.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590540 entries, 0 to 590539
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   TransactionID     590540 non-null  int64         
 1   TransactionAmt    590540 non-null  float64       
 2   ProductCD         590540 non-null  object        
 3   card1             590540 non-null  int64         
 4   addr1             590540 non-null  float64       
 5   transaction_time  590540 non-null  datetime64[ns]
 6   addr1_missing     590540 non-null  int64         
 7   time_diff         576987 non-null  float64       
 8   txn_per_hour      576987 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 45.1+ MB


In [84]:
txn['card1'].nunique()

13553

In [85]:
txn['txn_per_hour'] = txn['txn_per_hour'].fillna(0)
txn['time_diff'] = txn['time_diff'].fillna(0)

In [86]:
stats=txn.groupby('card1')['TransactionAmt'].agg(['mean','std'])

In [87]:
txn = txn.join(stats,on='card1')

In [88]:
txn['amount_deviation'] = (txn['TransactionAmt'] - txn['mean'])/txn['std']

Time Deviation

In [89]:
txn['hour']= txn['transaction_time'].dt.hour
hour_stats = txn.groupby('card1')['hour'].mean().rename('avg_txn_hour')
txn = txn.join(hour_stats, on='card1')
txn['time_deviation']= abs(txn['hour']- txn['avg_txn_hour'])

In [90]:
txn['prev_addr'] = txn.groupby('card1')['addr1'].shift(1)
txn['geo_mismatch'] = (txn['addr1'] != txn['prev_addr']).astype(int)

In [91]:
df = txn.copy()

In [92]:
df.drop(columns=['transaction_time','mean','std','avg_txn_hour'], inplace=True)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 590540 entries, 0 to 590539
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   TransactionID     590540 non-null  int64  
 1   TransactionAmt    590540 non-null  float64
 2   ProductCD         590540 non-null  object 
 3   card1             590540 non-null  int64  
 4   addr1             590540 non-null  float64
 5   addr1_missing     590540 non-null  int64  
 6   time_diff         590540 non-null  float64
 7   txn_per_hour      590540 non-null  float64
 8   amount_deviation  585689 non-null  float64
 9   hour              590540 non-null  int32  
 10  time_deviation    590540 non-null  float64
 11  prev_addr         576987 non-null  float64
 12  geo_mismatch      590540 non-null  int64  
dtypes: float64(7), int32(1), int64(4), object(1)
memory usage: 60.8+ MB


In [94]:
df['prev_addr'] = df['prev_addr'].fillna(-1)

In [95]:
df['amount_deviation'] = df['amount_deviation'].fillna(0)

In [96]:
df.describe()

D:\anaconda\Lib\site-packages\numpy\_core\_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,TransactionID,TransactionAmt,card1,addr1,addr1_missing,time_diff,txn_per_hour,amount_deviation,hour,time_deviation,prev_addr,geo_mismatch
count,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000,5.905400e+05,5.905400e+05,5.905400e+05,590540.000000,590540.000000,590540.000000,590540.000000
mean,3.282270e+06,135.027176,9898.734658,258.274247,0.111264,1.665192e+05,inf,NaN,13.861923,6.227719,252.480108,0.494469
std,1.704744e+05,239.162522,4901.170153,132.723241,0.314459,6.987376e+05,NaN,NaN,7.607152,4.088858,136.601103,0.499970
min,2.987000e+06,0.251000,1000.000000,-1.000000,0.000000,0.000000e+00,0.000000e+00,-inf,0.000000,0.000000,-1.000000,0.000000
25%,3.134635e+06,43.321000,6019.000000,184.000000,0.000000,8.430000e+02,5.361192e-02,-4.618588e-01,6.000000,2.691589,184.000000,0.000000
50%,3.282270e+06,68.769000,9678.000000,272.000000,0.000000,5.461000e+03,5.590062e-01,-2.837921e-01,16.000000,5.862178,269.000000,0.000000
75%,3.429904e+06,125.000000,14184.000000,327.000000,0.000000,5.551225e+04,3.498542e+00,9.222917e-02,20.000000,9.118421,325.000000,1.000000
max,3.577539e+06,31937.391000,18396.000000,540.000000,1.000000,1.559967e+07,inf,inf,23.000000,18.857143,540.000000,1.000000


In [97]:
df['amount_deviation'].max()

np.float64(inf)

In [98]:
df['amount_deviation'] = df['amount_deviation'].replace([np.inf, -np.inf], np.nan)

In [99]:
df['amount_deviation'] = df['amount_deviation'].fillna(0)

In [100]:
df['txn_per_hour'] = df['txn_per_hour'].replace([np.inf, -np.inf], np.nan)
df['txn_per_hour'] = df['txn_per_hour'].fillna(0)

In [101]:
df['amount_dev_cap'] = df['amount_deviation'].clip(-3, 3)
df['time_dev_cap'] = df['time_deviation'].clip(0, 6)
df['velocity_cap'] = df['txn_per_hour'].clip(0, 200)

In [102]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df[['amt_norm','time_norm','vel_norm']] = scaler.fit_transform(
    df[['amount_dev_cap','time_dev_cap','velocity_cap']]
)

In [103]:
df['risk_score'] = (
    0.5 * df['amt_norm'] +
    0.3 * df['vel_norm'] +
    0.2 * df['time_norm']
) * 100

In [104]:
df['risk_score'].describe()

count    590540.000000
mean         40.496861
std           9.875755
min           0.690278
25%          34.166915
50%          41.222718
75%          44.855553
max         100.000000
Name: risk_score, dtype: float64

In [105]:
def assign_risk_bucket(score):
    if score >= 80:
        return 'High Risk – Manual Review'
    elif score >= 50:
        return 'Medium Risk – Monitor'
    else:
        return 'Low Risk – Auto Clear'

df['risk_bucket'] = df['risk_score'].apply(assign_risk_bucket)

In [106]:
merchant_risk = (
    df.groupby('addr1')
       .agg(
           avg_risk_score=('risk_score', 'mean'),
           txn_count=('TransactionID', 'count'),
           high_risk_txn=('risk_bucket', lambda x: (x == 'High Risk – Manual Review').sum())
       )
       .reset_index()
)

In [107]:
merchant_risk['merchant_risk_bucket'] = merchant_risk['avg_risk_score'].apply(assign_risk_bucket)

In [115]:
merchant_risk[merchant_risk['avg_risk_score']>80]

,addr1,avg_risk_score,txn_count,high_risk_txn,merchant_risk_bucket


In [109]:
df.to_csv('transactions_scored.csv', index=False)
merchant_risk.to_csv('merchant_risk.csv', index=False)

In [110]:
df.to_excel('transactions_scored.xlsx', index=False)
merchant_risk.to_excel('merchant_risk.xlsx', index=False)

In [111]:
from IPython.display import FileLink
FileLink('transactions_scored.csv')

C:\Users\Harsh\Fraud Detection\transactions_scored.csv